In [1]:
import pandas
import os
import glob
from datetime import datetime, timedelta
import dask

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

In [2]:
data_folder = os.path.expanduser('~/localdatalake/tse_refined')

In [3]:
def aggregate(filename):
    df = pandas.read_csv(filename, sep=';')
    columns_agg = [x for x in df.columns if x.startswith('Quantidade')]
    columns_grouby = [x for x in df.columns if (x not in columns_agg) and (x not in ['Zona','Seção'])]
    df = df.groupby(columns_grouby, as_index=False).agg({x: 'sum' for x in columns_agg})
    return df
tipo = 'zona'
if tipo == 'secao':
    files = glob.glob(os.path.join(data_folder, 'perfil', 'PerfilSecao*_SP.csv.gz'))
elif tipo == 'zona':
    files = glob.glob(os.path.join(data_folder, 'perfil', 'PerfilZona*.csv.gz'))
df = pandas.concat((aggregate(x) for x in sorted(files)), copy=False, axis=0, ignore_index=True)

/home/tiagobbatalhao/Envs/DataScience/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




In [4]:
columns = [x for x in df.columns if not x.startswith('Quantidade')]
columns = ['FaixaEtária', 'UF']
for col in columns:
    df[col] = df[col].astype('category')

In [5]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14430399 entries, 0 to 14430398
Data columns (total 10 columns):
Ano                      int64
Escolaridade             int64
EstadoCivil              int64
FaixaEtária              category
Gênero                   int64
Município                int64
Quantidade               int64
QuantidadeDeficiência    float64
QuantidadeNomeSocial     float64
UF                       category
dtypes: category(2), float64(2), int64(6)
memory usage: 908.3 MB


In [6]:
df[['Quantidade']].describe()

,Quantidade
count,1.443040e+07
mean,1.223200e+02
std,1.825540e+03
min,1.000000e+00
25%,2.000000e+00
50%,9.000000e+00
75%,4.400000e+01
max,8.914880e+05


In [7]:
df['Ano'] = df['Ano'].apply(lambda x: x if x<=2018 else datetime.today().year)

## Evolução por gênero

In [8]:
plot_df = (
    df
    .groupby(['Ano', 'Gênero'], as_index=False)
    .agg({'Quantidade': 'sum'})
    .sort_values(by=['Ano', 'Gênero'])
)
plot_data = [
    go.Scatter(
        x=plot_df[plot_df['Gênero']==4]['Ano'],
        y=plot_df[plot_df['Gênero']==4]['Quantidade'],
        name='Mulheres',
        mode='lines+markers',
    ),
    go.Scatter(
        x=plot_df[plot_df['Gênero']==2]['Ano'],
        y=plot_df[plot_df['Gênero']==2]['Quantidade'],
        name='Homens',
        mode='lines+markers',
    ),
    go.Scatter(
        x=plot_df[plot_df['Gênero']==0]['Ano'],
        y=plot_df[plot_df['Gênero']==0]['Quantidade'],
        name='Desconhecido',
        mode='lines+markers',
    ),
]
plot_layout = dict(
    yaxis=dict(title='Pessoas', rangemode='tozero'),
    xaxis=dict(title='Ano'),
)
figure = go.Figure(data=plot_data, layout=plot_layout)
py.iplot(figure)

## Com ensino superior

In [9]:
plot_df = (
    df
    .assign(flag=lambda x: (x['Escolaridade'].isin([7,8])))
    .groupby(['Ano', 'flag'], as_index=False)
    .agg({'Quantidade': 'sum'})
#     .sort_values(by=['Ano', 'flag'])
)
plot_data = [
    go.Scatter(
        x=plot_df[plot_df['flag']]['Ano'],
        y=plot_df[plot_df['flag']]['Quantidade'],
        name='Com ensino superior',
        mode='lines+markers',
    ),
    go.Scatter(
        x=plot_df[~plot_df['flag']]['Ano'],
        y=plot_df[~plot_df['flag']]['Quantidade'],
        name='Sem ensino superior',
        mode='lines+markers',
    ),
]
plot_layout = dict(
    yaxis=dict(title='Pessoas', rangemode='tozero'),
    xaxis=dict(title='Ano'),
)
figure = go.Figure(data=plot_data, layout=plot_layout)
py.iplot(figure)

## Todas as escolaridades

In [10]:
plot_df = (
    df
    .groupby(['Ano', 'Escolaridade'], as_index=False)
    .agg({'Quantidade': 'sum'})
    .sort_values(by=['Ano', 'Escolaridade'])
)
plot_data = []
for i in range(0,9):
    plot_data += [
        go.Scatter(
            x=plot_df[plot_df['Escolaridade']==i]['Ano'],
            y=plot_df[plot_df['Escolaridade']==i]['Quantidade'],
            name='Escolaridade {}'.format(i),
            mode='lines+markers',
        ),
    ]
plot_layout = dict(
    yaxis=dict(title='Pessoas', rangemode='tozero'),
    xaxis=dict(title='Ano'),
)
figure = go.Figure(data=plot_data, layout=plot_layout)
py.iplot(figure)

## Estado civil

In [11]:
plot_df = (
    df
    .groupby(['Ano', 'EstadoCivil'], as_index=False)
    .agg({'Quantidade': 'sum'})
    .sort_values(by=['Ano', 'EstadoCivil'])
)
plot_data = []
for i in sorted(plot_df['EstadoCivil'].unique()):
    plot_data += [
        go.Scatter(
            x=plot_df[plot_df['EstadoCivil']==i]['Ano'],
            y=plot_df[plot_df['EstadoCivil']==i]['Quantidade'],
            name='EstadoCivil {}'.format(i),
            mode='lines+markers',
        ),
    ]
plot_layout = dict(
    yaxis=dict(title='Pessoas', rangemode='tozero'),
    xaxis=dict(title='Ano'),
)
figure = go.Figure(data=plot_data, layout=plot_layout)
py.iplot(figure)

## Faixa etária

In [12]:
def age_groups(s):
    split = s.split('-')
    to_int = lambda x: int(x) if len(x) else 120
    if len(split)==2:
        idade_min, idade_max = to_int(split[0]), to_int(split[1])
    else:
        idade_min, idade_max = to_int(split[0]), 120
    return tuple([idade_min, idade_max])

plot_df = df[df['Ano']>=2000]
plot_df['Idade'] = plot_df['FaixaEtária'].astype(str).apply(age_groups)

plot_df = (
    plot_df
    .groupby(['Ano', 'Idade'], as_index=False)
    .agg({'Quantidade': 'sum'})
    .sort_values(by=['Ano', 'Idade'])
)
plot_data = []
for i in sorted(plot_df['Idade'].unique()):
    plot_data += [
        go.Scatter(
            x=plot_df[plot_df['Idade']==i]['Ano'],
            y=plot_df[plot_df['Idade']==i]['Quantidade'],
            name='Idade {}'.format(i),
            mode='lines+markers',
        ),
    ]
plot_layout = dict(
    yaxis=dict(title='Pessoas', rangemode='tozero'),
    xaxis=dict(title='Ano'),
)
figure = go.Figure(data=plot_data, layout=plot_layout)
py.iplot(figure)

/home/tiagobbatalhao/Envs/DataScience/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [13]:
def age_groups(s):
    idade = int(s.split('-')[0])
    if idade == 0:
        return '0'
    if idade <= 24:
        group = '16-24'
    elif idade <= 34:
        group = '25-34'
    elif idade <= 44:
        group = '35-44'
    elif idade <= 59:
        group = '45-59'
    elif idade <= 69:
        group = '60-69'
    elif idade <= 79:
        group = '70-79'
    else:
        group = '80+'
    return group

plot_df = df[df['Ano']>=2000]
plot_df['Idade'] = plot_df['FaixaEtária'].astype(str).apply(age_groups)

plot_df = (
    plot_df
    .groupby(['Ano', 'Idade'], as_index=False)
    .agg({'Quantidade': 'sum'})
    .sort_values(by=['Ano', 'Idade'])
)
plot_data = []
for i in sorted(plot_df['Idade'].unique()):
    plot_data += [
        go.Scatter(
            x=plot_df[plot_df['Idade']==i]['Ano'],
            y=plot_df[plot_df['Idade']==i]['Quantidade'],
            name='Idade {}'.format(i),
            mode='lines+markers',
        ),
    ]
plot_layout = dict(
    yaxis=dict(title='Pessoas', rangemode='tozero'),
    xaxis=dict(title='Ano'),
)
figure = go.Figure(data=plot_data, layout=plot_layout)
py.iplot(figure)

/home/tiagobbatalhao/Envs/DataScience/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [14]:
def age_groups(s):
    idade = int(s.split('-')[0])
    if idade == 0:
        return '0'
    elif idade <= 34:
        group = '16-34'
    elif idade <= 59:
        group = '35-59'
    else:
        group = '60+'
    return group

plot_df = df[df['Ano']>=2000]
plot_df['Idade'] = plot_df['FaixaEtária'].astype(str).apply(age_groups)

plot_df = (
    plot_df
    .groupby(['Ano', 'Idade'], as_index=False)
    .agg({'Quantidade': 'sum'})
    .sort_values(by=['Ano', 'Idade'])
)
plot_data = []
cumulative = []
colors = ['yellow', 'red','blue', 'green']
for index, age in enumerate(sorted(plot_df['Idade'].unique())):
    cumulative.append(plot_df[plot_df['Idade']==age]['Quantidade'].values)
    anos = plot_df[plot_df['Idade']==age]['Ano']
    plot_data += [
        go.Bar(
            x=anos,
            y=cumulative[-1],
            name='Idade {}'.format(age),
            marker=dict(color=colors[index]),
        ),
    ]
    plot_data += [
        go.Scatter(
            x=anos,
            y=sum(cumulative),
            marker=dict(color=colors[index]),
#             name='Idade {}'.format(i),
            mode='lines',
            showlegend=False,
        ),
    ]
plot_layout = dict(
    barmode='stack',
    yaxis=dict(title='Pessoas', rangemode='tozero'),
    xaxis=dict(title='Ano'),
)
figure = go.Figure(data=plot_data, layout=plot_layout)
py.iplot(figure)

/home/tiagobbatalhao/Envs/DataScience/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [33]:
def age_groups(s):
    idade = int(s.split('-')[0])
    if idade == 0:
        return '0'
    elif idade <= 24:
        group = '16-24'
    elif idade <= 34:
        group = '25-34'
    elif idade <= 44:
        group = '35-44'
    elif idade <= 59:
        group = '45-59'
    else:
        group = '60+'
    return group

plot_df = df[df['Ano']>=2000]
plot_df['Idade'] = plot_df['FaixaEtária'].astype(str).apply(age_groups)

plot_df = (
    plot_df
    .groupby(['Ano', 'Idade'], as_index=False)
    .agg({'Quantidade': 'sum'})
    .sort_values(by=['Ano', 'Idade'])
)
groups = ['16-24', '25-34', '35-44', '45-59', '60+']
plot_data = [
    go.Bar(
        x=plot_df.query("Idade != '0'").query("Ano == {}".format(ano))['Quantidade'],
        y=list(range(len(groups))),
        name=str(ano),
#         marker=dict(color='blue'),
        orientation='h',
    )
    for ano in [2018, 2012, 2006, 2000]
]
plot_layout = dict(
#     barmode='group',
    yaxis=dict(title='Faixa etária', tickvals=list(range(len(groups))), ticktext=groups),
    xaxis=dict(title='Pessoas'),
)
figure = go.Figure(data=plot_data, layout=plot_layout)
py.iplot(figure)

/home/tiagobbatalhao/Envs/DataScience/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

